In [2]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  # accessor needed to parse crs
import calendar
import argparse
import seaborn as sns
from shapely.geometry import Point
import geopandas
from geodatasets import get_path
import glob

In [31]:

# vegc_paths = glob.glob(r'/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_'+var+'{12530}/*')
# soilc_paths = glob.glob(r'/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_'+var+'12530*')
# forest_paths = vegc_paths + soilc_paths
# forest_paths
# var = '[GPP|NPP]'
# number = [12530, 13783, 15036, 16289, 17542]
# var = '[SOILORGC|VEGC]'
# soilc_paths = glob.glob(r'/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_'+var+'*')
# soilc_paths
files = [
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC12530.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC13783.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC13783.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC15036.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC15036.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC16289.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC16289.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC17542.csv'   
, '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC17542.csv'   

]
files
# print(len(vegc_paths), ' ', len(soilc_paths))

['/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC12530.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC13783.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC13783.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC15036.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC15036.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC16289.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC16289.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC17542.csv',
 '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC17542.csv']

In [3]:
test=pd.read_csv( '/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv'  )

In [4]:
test

,lon,lat,variable,pft,current_veg,year,month,stand_age_interval_min,cohort_area,land_area,...,temp_weight,value_weight,forest_type,Area,prec,par,tair,nirr,vpr,ws10
0,-73.0,-46.0,VEGC,forest,10,1501,Apr,3000,2134,2157,...,225947.92,105.880000,Temperate Deciduous Forests,2157,172.1,39.1,4.7,61.1,7.6,6.1
1,-73.0,-46.0,VEGC,forest,10,1501,Aug,3000,2134,2157,...,223771.24,104.860000,Temperate Deciduous Forests,2157,161.7,33.0,2.2,51.8,6.5,5.7
2,-73.0,-46.0,VEGC,forest,10,1501,Dec,3000,2134,2157,...,224880.92,105.380000,Temperate Deciduous Forests,2157,140.6,122.2,7.0,206.9,8.5,6.5
3,-73.0,-46.0,VEGC,forest,10,1501,Feb,3000,2134,2157,...,226161.32,105.980000,Temperate Deciduous Forests,2157,143.1,95.6,7.8,158.6,9.0,6.5
4,-73.0,-46.0,VEGC,forest,10,1501,Jan,3000,2134,2157,...,225777.20,105.800000,Temperate Deciduous Forests,2157,184.9,96.3,7.8,153.5,9.2,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16809895,-70.0,58.0,VEGC,forest,6,2014,Mar,3000,1619,3252,...,3936056.43,2431.165182,Boreal Woodlands,1626,65.4,93.7,-9.8,208.1,2.5,0.5
16809896,-70.0,58.0,VEGC,forest,6,2014,May,3000,1619,3252,...,3962938.26,2447.769154,Boreal Woodlands,1626,63.9,124.0,8.4,230.8,8.9,1.2
16809897,-70.0,58.0,VEGC,forest,6,2014,Nov,3000,1619,3252,...,3960748.70,2446.416739,Boreal Woodlands,1626,27.3,11.7,-16.7,18.2,1.6,1.1
16809898,-70.0,58.0,VEGC,forest,6,2014,Oct,3000,1619,3252,...,3978662.34,2457.481371,Boreal Woodlands,1626,42.6,20.5,-9.1,31.5,2.9,0.5


In [33]:
all_grids = pd.DataFrame( columns = [ 'year', 'lon', 'lat', 'variable','current_veg', 'stand_age'
       'cohort_area', 'land_area', 'value',  'temp', 'value_weight'])

# a
for file in files:
    grids = pd.read_csv(file)
    
    print(grids.value.mean())
    print(grids.value.max())

    if len(grids) >0:
        aggregations = {
         'cohort_area':'sum',
         'land_area':'sum',
         'value':'mean',
            'temp':'sum'
        }
        grids['temp'] = grids['value_weight'] * grids['cohort_area'] 


        grids = grids.groupby([
               'year', 'lon', 'lat', 'variable','current_veg']).agg(
        aggregations
        )


        # all_grids =  all_grids.reset_index()

        grids['value_weight'] = grids['temp']/grids['cohort_area']
        grids = grids.reset_index()

        all_grids = pd.concat([all_grids, grids])
        
        
    print(file)
    print(len(all_grids))
all_grids.to_csv('amazon_loop_output_vegc_soilc_historical.csv')    

11326.046899097038
37429.41
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC12530.csv
352090
5766.03361230075
99053.99
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC12530.csv
704180
11532.777738814313
60236.68
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC13783.csv
1005384
9695.934862885195
363131.8358333334
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC13783.csv
1306588
7999.481325613213
31485.755
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC15036.csv
1782552
3885.1199655640257
33347.96
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_SOILORGC15036.csv
2258516
6667.4740061878665
32440.0
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts/forests_VEGC16289.csv
2658922
3542.0595119547884
22261.41
/home/smmrrr/TEM_output_processed/historical_run/forest_lon_lat_pfts